In [11]:
from plotly.graph_objects import Scatter, Figure
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv('trusted-for-alternative-fixed.csv')

In [13]:
areas = ['city', 'minsk_suburb', 'minsk_village', 'capital', 'village', 'town_below100', 'town_over100', 'embassy']
candidates = ['against', 'cherechen', 'corrupted', 'dmitriyev', 'kanopatskaja', 'tihanovkaja']

In [14]:
def add_exp_off_trace(fig, data, coeff, name):
    protest_expected = (data['registered'] - data['lukashenko_registered'] - data['ignore_registered']) * coeff
    protest_official = sum(data[f'{c}_officialVotes'] for c in candidates)  

    fig.add_trace(Scatter(
        x=protest_expected, 
        y=protest_official,
        mode='markers',
        name=name,
        text=[
            f'{ps_id} ({zubr_id})' 
            for ps_id, zubr_id in zip(data['id'], data['zubr_id'])
        ]
    ))


def expected_vs_official_detailed(area, region, coeff):
    fig = Figure(layout_title=f'Expected vs Official ({area} - {region})')

    mask = (df['area'] == area) & (df['region'] == region)  
    
    add_exp_off_trace(
        fig, 
        df[mask & df['wrong-voters-number']], 
        coeff, 
        'wrong voters'
    )
    add_exp_off_trace(
        fig, 
        df[mask & ~df['wrong-voters-number'] & (df['observers'] == 0)], 
        coeff, 
        'no-observers'
    )
    add_exp_off_trace(
        fig, 
        df[mask & ~df['wrong-voters-number'] & df['major-violations'] & (df['observers'] > 0)], 
        coeff, 
        'major'
    )
    add_exp_off_trace(
        fig, 
        df[mask & ~df['wrong-voters-number'] & ~df['major-violations'] & df['minor-violations'] & (df['observers'] > 0)], 
        coeff, 
        'minor'
    )
    add_exp_off_trace(
        fig, 
        df[mask & ~df['wrong-voters-number'] & ~df['major-violations'] & ~df['minor-violations'] & (df['observers'] > 0)], 
        coeff,
        'ok'
    )

    data = df[mask]        
    protest_expected = (data['registered'] - data['lukashenko_registered'] - data['ignore_registered']) * coeff
        
    fig.add_trace(Scatter(
        x=protest_expected, 
        y=protest_expected,
        mode='lines',
        name=f'Expected',    
    ))

    fig.update_xaxes(title='Expected')
    fig.update_yaxes(title='Official')


    fig.show()

In [15]:
expected_vs_official_detailed('capital', 7, 2.069)

In [16]:
def add_photo_trace(fig, data, coeff, name):
    tih_expected = data['tihanovkaja_photoVoices'] * coeff
    tih_official = data['tihanovkaja_officialVotes']

    fig.add_trace(Scatter(
        x=tih_expected, 
        y=tih_official,
        mode='markers',
        name=name,
        text=[
            f'{ps_id} ({zubr_id})' 
            for ps_id, zubr_id in zip(data['id'], data['zubr_id'])
        ]
    ))


def photo_detailed(area, region, coeff):
    fig = Figure(layout_title=f'Tihanovkaja Expected vs Official ({area} - {region})')

    mask = (df['area'] == area) & (df['region'] == region)  
    
    add_photo_trace(
        fig, 
        df[mask & df['wrong-voters-number']], 
        coeff, 
        'wrong voters'
    )
    add_photo_trace(
        fig, 
        df[mask & ~df['wrong-voters-number'] & (df['observers'] == 0)], 
        coeff, 
        'no-observers'
    )
    add_photo_trace(
        fig, 
        df[mask & ~df['wrong-voters-number'] & df['major-violations'] & (df['observers'] > 0)], 
        coeff, 
        'major'
    )
    add_photo_trace(
        fig, 
        df[mask & ~df['wrong-voters-number'] & ~df['major-violations'] & df['minor-violations'] & (df['observers'] > 0)], 
        coeff, 
        'minor'
    )
    add_photo_trace(
        fig, 
        df[mask & ~df['wrong-voters-number'] & ~df['major-violations'] & ~df['minor-violations'] & (df['observers'] > 0)], 
        coeff,
        'ok'
    )

    data = df[mask]        
    tih_expected = data['tihanovkaja_photoVoices'] * coeff
        
    fig.add_trace(Scatter(
        x=tih_expected, 
        y=tih_expected,
        mode='lines',
        name=f'Expected',    
    ))

    fig.update_xaxes(title='Expected')
    fig.update_yaxes(title='Official')


    fig.show()

In [24]:
photo_detailed('village', 1, 7.760)

In [10]:
photo_detailed('capital', 7, 2.557)

In [232]:
draw_detailed_chart('capital', 7)

In [122]:
expected_vs_official('village', 2.973)

In [63]:
fig = Figure()

mask = (
    (df['officialVotes'] > 0) &
    (df['officialVoters'] > 0)
)

for area in areas:
    data = df[mask & (df['area'] == area)]
    fig.add_trace(Scatter(
        x=np.arange(len(data)), 
        y=data['officialVoters'],
        mode='markers',
        name=area,
        text=data['region'],
    ))

fig.update_xaxes(title='Poll stations')
fig.update_yaxes(title='Voters')


fig.show()

In [66]:
def turnout_to_luk(area):
    fig = Figure(layout_title=f'Turnout to Lukashenko votes ({area})')

    mask = (
        (df['area'] == area) &
        (df['officialVotes'] > 0) &
        (df['officialVoters'] > 0)
    )
    
    for region in range(1, 9):
        data = df[mask & (df['region'] == region)]
        
        fig.add_trace(Scatter(
            x=data['officialVotes'] / data['officialVoters'], 
            y=data['lukashenko_officialVotes'] / data['officialVotes'],
            mode='markers',
            name=f'{region}',
            text=data['id'],
        ))
        
    fig.update_xaxes(title='Turnout')
    fig.update_yaxes(title='Luk %')


    fig.show()

In [95]:
turnout_to_luk('village')